In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [ ]:
# train data
raw_data = pd.read_excel("Hackathon Data.xlsx")

## Reading Holdout scores

In [ ]:
costran_df = pd.read_csv('croston_holdout.csv').drop(columns = ['Unnamed: 0'])
xbg_results = pd.read_csv('holdout rmse xgb.csv')
xgb_dict = dict((k,v) for k,v in zip(xbg_results['sku'].values,xbg_results['rmse'].values))

## Checking RMSE for croston and XGBoost at sku level to select best strategy

In [ ]:
ensemble_results = pd.DataFrame()
for idx,sku_id in enumerate(list(costran_df['sku'].unique())) :

    df = costran_df[costran_df['sku'] == sku_id]
    
    mean_sales = raw_data[raw_data['Encoded_SKU_ID'] == sku_id][:-7]['DAILY_UNITS'].mean()
    median_sales = raw_data[raw_data['Encoded_SKU_ID'] == sku_id][:-7]['DAILY_UNITS'].median()
    min_sales = raw_data[raw_data['Encoded_SKU_ID'] == sku_id][:-7]['DAILY_UNITS'].min()
    max_sales = raw_data[raw_data['Encoded_SKU_ID'] == sku_id][:-7]['DAILY_UNITS'].max()
    
    costran_rmse  = mean_squared_error(df["pred"],df["DAILY_UNITS"],squared=False)
    mean_rmse = mean_squared_error(df["DAILY_UNITS"].values,[mean_sales]*df.shape[0],squared=False)
    
    ensemble_results.loc[idx,'sku'] = sku_id
    ensemble_results.loc[idx,'mean_sales'] = mean_sales
    ensemble_results.loc[idx,'median_sales'] = median_sales
    ensemble_results.loc[idx,'min_sales'] = min_sales
    ensemble_results.loc[idx,'max_sales'] = max_sales
    
    ensemble_results.loc[idx,'costran_rmse'] = costran_rmse
    ensemble_results.loc[idx,'xgb_rmse'] = xgb_dict[sku_id]
    ensemble_results.loc[idx,'baseline_rmse'] = mean_rmse
    
    
ensemble_results  

In [ ]:
ensemble_results['min_rmse'] = ensemble_results.apply(lambda r : min(r['costran_rmse'],\
                                                                     r['xgb_rmse'],r['baseline_rmse']),axis=1)

def func(row) :
    if row['min_rmse'] == row['costran_rmse']:
        return 'costran'
    elif row['min_rmse'] == row['xgb_rmse']:
        return 'xgboost'
    else:
        return 'baseline'

ensemble_results['model'] = ensemble_results.apply(lambda r : func(r),axis=1)
ensemble_results.drop(columns=['min_rmse'],inplace=True)

ensemble_dict = dict((k,v) for k,v in zip(ensemble_results['sku'].values,ensemble_results['model'].values))

## Reading Validation Predictions

In [ ]:
# validation scores read
costran_score = pd.read_csv('Croston_validation_pred.csv').drop(columns = ['Unnamed: 0'])
xgboost_score = pd.read_csv('validation_latest_prediction_xgboost.csv').drop(columns = ['Unnamed: 0'])
xgboost_score['SALES_DATE'] = pd.to_datetime(xgboost_score['SALES_DATE'],infer_datetime_format=True)
costran_score['dt'] = pd.to_datetime(costran_score['dt'],infer_datetime_format=True)
mean_score_dict = dict((k,v) for k,v in zip(ensemble_results['sku'].values,ensemble_results['mean_sales'].values))
# non-overlapping sku - extra in xgb
extras = list(set(list(costran_score['Encoded_SKU_ID'].unique()))^set(list(xgboost_score['sku_id'].unique())))
xgboost_score = xgboost_score[~xgboost_score['sku_id'].isin(extras)]
len(costran_score),len(xgboost_score)

In [ ]:
all_scores = pd.merge(costran_score[['sku','dt','pred']].rename(columns={'pred':'costran_pred'}),\
                     xgboost_score[['sku_id','SALES_DATE','pred','DAILY_UNITS']].rename(columns={'sku_id':'sku','pred':'xgb_pred','SALES_DATE':'dt'}),\
                      on=['sku','dt'])

## Calculating forecasts based on Ensemble Strategy

In [ ]:
def final_forecast_ensemble(row):
    if ensemble_dict[row['sku']] == 'xgboost':
        return round(row['xgb_pred'])
#     elif ensemble_dict[row['Encoded_SKU_ID']] == 'baseline':
#         return round(mean_score_dict[row['Encoded_SKU_ID']])
    return round(row['costran_pred'],0)
    
all_scores['final_forecast'] = all_scores.apply(lambda r : final_forecast_ensemble(r),axis=1)

# Final Metrics on Validation Set

In [ ]:
from sklearn import metrics
mae = metrics.mean_absolute_error(round(all_scores['final_forecast'],0), all_scores['DAILY_UNITS'])
mse = metrics.mean_squared_error(round(all_scores['final_forecast'],0), all_scores['DAILY_UNITS'])
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(round(all_scores['final_forecast'],0), all_scores['DAILY_UNITS'])

print("Results of FINAL MODEL")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)